In [ ]:
from __init__ import *

In [ ]:
signal_ = []

for i, dict in enumerate(SETTING["model_data_"]):

    signal = pd.read_csv("../output/signal/{}.tsv".format(i), sep="\t", index_col=0)

    signal.columns.name = dict["axis_1_name"]

    signal_.append(signal)

In [ ]:
if SETTING["model_mode"] == "range":

    axis = 0

    mode = "hs"

elif SETTING["model_mode"] == "deep":

    axis = 1

    mode = "ws"

signal_ = kraft.table.sync_axis(signal_, axis)

In [ ]:
for r in SETTING["resolution_"]:

    w_, h_, error_ = kraft.matrix_factorization.factorize_matrix(
        tuple(signal.to_numpy() for signal in signal_), mode, r
    )

    template = "Factor {}.{{}}".format(r)

    factor_label_ = pd.Index(data=(template.format(i) for i in range(r)), name="Factor")

    w_ = tuple(
        pd.DataFrame(data=w, index=signal.index, columns=factor_label_)
        for w, signal in zip(w_, signal_)
    )

    h_ = tuple(
        pd.DataFrame(data=h, index=factor_label_, columns=signal.columns)
        for h, signal in zip(h_, signal_)
    )

    directory_path = "../output/compression_r{}/".format(r)

    kraft.path.path(directory_path)

    template = "{}{{}}_{{}}.tsv".format(directory_path)

    for i, w in enumerate(w_):

        w.to_csv(template.format("w", i), sep="\t")

    for i, h in enumerate(h_):

        h.to_csv(template.format("h", i), sep="\t")

    kraft.matrix_factorization.plot_matrix_factorization(
        w_, h_, error_, directory_path=directory_path
    )